# User Labeling

## Présentation
Le script en question a pour objectif de labéliser les utilisateurs en deux catégories distinctes : ceux qui sont considérés comme atypiques (avec un label à 1) et ceux qui ne le sont pas (avec un label à 0). 

Les utilisateurs jugés atypiques pourront être identifiés grâce à des comportements ou des actions qui sortent de l'ordinaire

En labélisant les utilisateurs de cette manière, le script peut permettre de mieux comprendre les comportements des utilisateurs.

Comment fonctionne-t-il ?
1. Pour chaque attribut des utilisateurs, on calcul la valeur minimale dont possède **seulement 10% de la population** des utilisateurs avec une valeur maximale. Ces valeurs correspondent donc au 10% des valeurs les plus élevées que les utilisateurs possèdes.
2. Attribution d'un score de suspicion pour chaque utilisateur. Pour ce faire, on utilise toutes les valeurs trouvées précédemment et on les compare avec les valeurs de l'utilisateur. Plus l'utilisateur possède de valeur supérieure aux valeurs précédente, plus il est atypique.
3. Si se score dépasse 3, alors il est considéré comme atypique.

## Le code
On importe les différentes bibliothèques nécessaires.

In [ ]:
import pymongo
import math

Connexion à la base de données MongoDB

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
user_collection = db["users"]
user_labeled_collection = db["users_labeled"]

On supprime toute la collection des utilisateurs labellisés pour supprimer par la même occasion les données qu'elle contient.

In [ ]:
user_labeled_collection.drop()

On définit les champs sur lesquels on va déterminer si un utilisateur est atypique ou non

In [ ]:
field_to_analyse = ['hashtag_frequency', 'favourites_count', 'friends_count', 'followers_count', 'ratio_friends_followers', 'tweet_frequency', 'Ap', 'ratio_sensitive_tweets', 'ratio_punctuation_tweets', 'visibility']

### Calcul des valeurs minimales de suspicion (10% de la population)

In [ ]:
percentage = 8
BATCH_SIZE = 100_000

total_documents = user_collection.count_documents({})
num_batches = math.ceil(total_documents / BATCH_SIZE)
nb_values_to_keep = math.ceil(total_documents * percentage / 100)

max_values = {field: [] for field in field_to_analyse}

for page in range(num_batches):
    print('Page n°', page)
    users_batch = user_collection.find({}).skip(page * BATCH_SIZE).limit(BATCH_SIZE)
    for user in users_batch:
        for field in field_to_analyse:
            max_values[field].append(user[field])
    
    for key, values in max_values.items():
        max_values[key] = sorted(values)[-nb_values_to_keep:]

res = {}
for key, values in max_values.items():
    res[key] = values[0]

affichage des valeur minimales de suspicions

In [ ]:
res

### Calcul du score de suspicion et du label

Définition de la fonction qui détermine le score de suspicion d'un utilisateur

In [ ]:
def get_suspicious_score(user):
    suspicious_fields = []
    for key, value in res.items():
        if user[key] > value:
            suspicious_fields.append(key)
    return (len(suspicious_fields), suspicious_fields)

Définition de la fonction qui détermine le label d'un utilisateur enfocntion de on score de suspicion

In [ ]:
def get_label(user):
    suspicious_score, suspicious_fields = get_suspicious_score(user)
    if suspicious_score >= 3:
        return (suspicious_score, suspicious_fields, 1)
    else:
        return (suspicious_score, suspicious_fields, 0)

On insérer dans mongodb les utilisateurs avec leur label calculé

In [ ]:
BATCH_SIZE = 100_000
page = 0
while True:
    print(f"Page n°{page}")
    users = user_collection.find({}).skip(page * BATCH_SIZE).limit(BATCH_SIZE)
    
    users_with_label = []
    for user in users:
        suspicious_score, suspicious_fields, label = get_label(user)
        user['label'] = label
        user['suspicious_score'] = suspicious_score
        user['suspicious_fields'] = suspicious_fields
        users_with_label.append(user)
    
    if(len(users_with_label) == 0):
        print('FIN')
        break

    user_labeled_collection.insert_many(users_with_label)

    page += 1

## Quelques statistiques

Regarder la répartition des points de suspicion

In [ ]:
res = user_labeled_collection.aggregate([
    {
        "$group": {
            "_id": "$suspicious_score",
            "count": { "$sum": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

docs = []
for doc in res:
    print(doc)
    docs.append(doc)

Regarder la répartition des labels atypique ou non

In [ ]:
res = user_labeled_collection.aggregate([
    {
        "$group": {
            "_id": "$label",
            "count": { "$sum": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

docs = []
for doc in res:
    print(doc)
    docs.append(doc)